# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [5]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

# Carregar os dados 
df = pd.read_csv('previsao_de_renda.csv').dropna()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12466 entries, 0 to 14999
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             12466 non-null  int64  
 1   data_ref               12466 non-null  object 
 2   index                  12466 non-null  int64  
 3   sexo                   12466 non-null  object 
 4   posse_de_veiculo       12466 non-null  bool   
 5   posse_de_imovel        12466 non-null  bool   
 6   qtd_filhos             12466 non-null  int64  
 7   tipo_renda             12466 non-null  object 
 8   educacao               12466 non-null  object 
 9   estado_civil           12466 non-null  object 
 10  tipo_residencia        12466 non-null  object 
 11  idade                  12466 non-null  int64  
 12  tempo_emprego          12466 non-null  float64
 13  qt_pessoas_residencia  12466 non-null  float64
 14  mau                    12466 non-null  bool   
 15  renda  

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [11]:
reg2 = smf.ols('''np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + 
                C(tipo_renda, Treatment(reference="Assalariado")) + 
                C(educacao, Treatment(reference="Secundário")) + 
                C(estado_civil, Treatment(reference="Casado")) + 
                idade + tempo_emprego + qt_pessoas_residencia''', data=df).fit()
print(reg2.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     363.0
Date:                Sun, 28 Dec 2025   Prob (F-statistic):               0.00
Time:                        21:30:06   Log-Likelihood:                -13569.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12407   BIC:                         2.733e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------

In [8]:
#exercício 1

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 1. Carregar os dados 
df = pd.read_csv('previsao_de_renda.csv')

# 2. Limpeza: Remover nulos e colunas que não são preditoras logo de início
df_limpo = df.dropna().drop(columns=['Unnamed: 0', 'id_cliente', 'data_ref'], errors='ignore')

# 3. Criar variáveis dummy para as colunas de texto (sexo, educacao, etc)
df_dummies = pd.get_dummies(df_limpo, drop_first=True)

# 4. Definir X (explicativas) e y (alvo com log)
X = df_dummies.drop(columns=['renda'])
y = np.log(df_dummies['renda'])

# 5. Divisão em Treino (75%) e Teste (25%) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print("Exercício 1 finalizado com sucesso!")
print(f"Variáveis explicativas: {X.shape[1]}")
print(f"Amostras de treino: {len(X_train)}")

Exercício 1 finalizado com sucesso!
Variáveis explicativas: 26
Amostras de treino: 9349


In [9]:
#exercício 2
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

# Lista de alphas
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
resultados_ridge = []

print("Avaliando Regressão Ridge:")
print("-" * 30)

for a in alphas:
    # 1. Instanciar e treinar o modelo
    # Nota: alpha=0 no Ridge é equivalente ao OLS comum
    modelo_ridge = Ridge(alpha=a)
    modelo_ridge.fit(X_train, y_train)
    
    # 2. Fazer a previsão na base de TESTE
    y_pred = modelo_ridge.predict(X_test)
    
    # 3. Calcular o R²
    r2 = r2_score(y_test, y_pred)
    
    # Guardar o resultado para comparação
    resultados_ridge.append({'alpha': a, 'R2_teste': r2})
    
    print(f"Alpha: {a:<6} | R² Teste: {r2:.4f}")

# Identificando o melhor modelo
melhor_ridge = max(resultados_ridge, key=lambda x: x['R2_teste'])
print("-" * 30)
print(f"O melhor modelo Ridge foi com alpha = {melhor_ridge['alpha']} (R² = {melhor_ridge['R2_teste']:.4f})")

Avaliando Regressão Ridge:
------------------------------
Alpha: 0      | R² Teste: 0.2323
Alpha: 0.001  | R² Teste: 0.2323
Alpha: 0.005  | R² Teste: 0.2323
Alpha: 0.01   | R² Teste: 0.2323
Alpha: 0.05   | R² Teste: 0.2323
Alpha: 0.1    | R² Teste: 0.2323
------------------------------
O melhor modelo Ridge foi com alpha = 0.1 (R² = 0.2323)


In [10]:
#exercício 3

from sklearn.linear_model import Lasso

# Mesma lista de alphas
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
resultados_lasso = []

print("Avaliando Regressão LASSO:")
print("-" * 30)

for a in alphas:
    # 1. Instanciar e treinar o modelo
    
    if a == 0:
        modelo_lasso = Ridge(alpha=0) # Equivalente ao OLS
    else:
        modelo_lasso = Lasso(alpha=a)
        
    modelo_lasso.fit(X_train, y_train)
    
    # 2. Previsão e R² na base de TESTE
    y_pred = modelo_lasso.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    resultados_lasso.append({'alpha': a, 'R2_teste': r2})
    print(f"Alpha: {a:<6} | R² Teste: {r2:.4f}")

# Melhor LASSO
melhor_lasso = max(resultados_lasso, key=lambda x: x['R2_teste'])
print("-" * 30)
print(f"O melhor modelo LASSO foi com alpha = {melhor_lasso['alpha']} (R² = {melhor_lasso['R2_teste']:.4f})")

# Comparação final solicitada no item 3
print(f"\nComparação:")
print(f"Melhor Ridge: {melhor_ridge['R2_teste']:.4f}")
print(f"Melhor LASSO: {melhor_lasso['R2_teste']:.4f}")

Avaliando Regressão LASSO:
------------------------------
Alpha: 0      | R² Teste: 0.2323
Alpha: 0.001  | R² Teste: 0.2343
Alpha: 0.005  | R² Teste: 0.2337
Alpha: 0.01   | R² Teste: 0.2304
Alpha: 0.05   | R² Teste: 0.1996
Alpha: 0.1    | R² Teste: 0.1486
------------------------------
O melhor modelo LASSO foi com alpha = 0.001 (R² = 0.2343)

Comparação:
Melhor Ridge: 0.2323
Melhor LASSO: 0.2343


In [14]:
#exercício 4
from sklearn.linear_model import LinearRegression

# 1. Identificar os nomes reais das colunas que queremos
# Vamos buscar colunas que contenham as palavras-chave para não errar o nome
col_sexo = [c for c in X_train.columns if 'sexo' in c][0]
col_imovel = [c for c in X_train.columns if 'posse_de_imovel' in c][0]

colunas_stepwise = [col_sexo, col_imovel, 'idade', 'tempo_emprego']

print(f"Colunas identificadas para o Stepwise: {colunas_stepwise}")

# 2. Selecionar e treinar
X_train_step = X_train[colunas_stepwise]
X_test_step = X_test[colunas_stepwise]

modelo_stepwise = LinearRegression()
modelo_stepwise.fit(X_train_step, y_train)

# 3. Previsão e R²
y_pred_stepwise = modelo_stepwise.predict(X_test_step)
r2_stepwise = r2_score(y_test, y_pred_stepwise)

print("-" * 30)
print(f"R² Stepwise (Teste): {r2_stepwise:.4f}")
print("-" * 30)

# Coeficientes para comparação no Passo 5
for col, coef in zip(colunas_stepwise, modelo_stepwise.coef_):
    print(f"{col}: {coef:.4f}")

Colunas identificadas para o Stepwise: ['sexo_M', 'posse_de_imovel', 'idade', 'tempo_emprego']
------------------------------
R² Stepwise (Teste): 0.2205
------------------------------
sexo_M: 0.4760
posse_de_imovel: 0.0953
idade: 0.0045
tempo_emprego: 0.0478


In [15]:
#Exercício 5 

print("Comparação Final dos Modelos (Base de Teste):")
print(f"1. R² Ridge:    {0.2323:.4f}")
print(f"2. R² LASSO:    {0.2343:.4f}")
print(f"3. R² Stepwise: {0.2205:.4f}")

# Análise:
analise = """
Avaliação dos Resultados:
O modelo LASSO (alpha=0.001) apresentou o melhor desempenho na base de teste (R² = 0.2343), 
superando levemente o Ridge e o Stepwise. 

Diferenças observadas:
- O Ridge manteve todas as variáveis, mas 'encolheu' seus coeficientes para lidar com a multicolinearidade.
- O LASSO foi o mais eficiente pois, além de regularizar, realizou uma seleção de variáveis 
  removendo ruídos que o modelo completo (Ridge) ainda considerava.
- O Stepwise, embora seja o modelo mais simples e fácil de interpretar, apresentou o menor R², 
  indicando que a exclusão manual de certas variáveis pode ter omitido informações relevantes 
  para a previsão.

Conclusão: O melhor modelo para este problema é o LASSO com alpha 0.001.
"""
print(analise)

Comparação Final dos Modelos (Base de Teste):
1. R² Ridge:    0.2323
2. R² LASSO:    0.2343
3. R² Stepwise: 0.2205

Avaliação dos Resultados:
O modelo LASSO (alpha=0.001) apresentou o melhor desempenho na base de teste (R² = 0.2343), 
superando levemente o Ridge e o Stepwise. 

Diferenças observadas:
- O Ridge manteve todas as variáveis, mas 'encolheu' seus coeficientes para lidar com a multicolinearidade.
- O LASSO foi o mais eficiente pois, além de regularizar, realizou uma seleção de variáveis 
  removendo ruídos que o modelo completo (Ridge) ainda considerava.
- O Stepwise, embora seja o modelo mais simples e fácil de interpretar, apresentou o menor R², 
  indicando que a exclusão manual de certas variáveis pode ter omitido informações relevantes 
  para a previsão.

Conclusão: O melhor modelo para este problema é o LASSO com alpha 0.001.



In [16]:
# 1. Criando uma cópia para não afetar os dados originais
X_train_enhanced = X_train.copy()
X_test_enhanced = X_test.copy()

# 2. Transformação Não-Linear: Tempo de Emprego ao quadrado
# Isso ajuda a capturar rendimentos decrescentes ao longo do tempo
X_train_enhanced['tempo_emprego_2'] = X_train_enhanced['tempo_emprego'] ** 2
X_test_enhanced['tempo_emprego_2'] = X_test_enhanced['tempo_emprego'] ** 2

# 3. Interação: Sexo e Tempo de Emprego
X_train_enhanced['interacao_sexo_tempo'] = X_train_enhanced['sexo_M'] * X_train_enhanced['tempo_emprego']
X_test_enhanced['interacao_sexo_tempo'] = X_test_enhanced['sexo_M'] * X_test_enhanced['tempo_emprego']

# 4. Treinando o modelo "Turbinado" com Ridge (para evitar que as novas colunas gerem overfitting)
modelo_melhorado = Ridge(alpha=0.001)
modelo_melhorado.fit(X_train_enhanced, y_train)

# 5. Avaliação
y_pred_enhanced = modelo_melhorado.fit(X_train_enhanced, y_train).predict(X_test_enhanced)
r2_enhanced = r2_score(y_test, y_pred_enhanced)

print("Exercício 6 - Modelo Incrementado:")
print("-" * 35)
print(f"R² Antigo (Melhor LASSO): 0.2343")
print(f"R² Novo (Com Transformações): {r2_enhanced:.4f}")
print("-" * 35)

if r2_enhanced > 0.2343:
    print("Sucesso! As transformações melhoraram o poder preditivo do modelo.")

Exercício 6 - Modelo Incrementado:
-----------------------------------
R² Antigo (Melhor LASSO): 0.2343
R² Novo (Com Transformações): 0.2322
-----------------------------------


In [19]:
#exercício 7 

#  Árvore de Regressão 
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

# 1. Instanciando e treinando a árvore
modelo_arvore = DecisionTreeRegressor(max_depth=5, random_state=42)
modelo_arvore.fit(X_train, y_train)

# 2. Fazendo a previsão na base de TESTE
y_pred_arvore = modelo_arvore.predict(X_test)

# 3. Cálculos de Mediana 
# Usamos np.exp() para converter o LOG da renda de volta para REAIS
mediana_real = np.median(np.exp(y_test))
mediana_predita = np.median(np.exp(y_pred_arvore))

# 4. Avaliação de métricas (R² e MAE)
r2_arvore = r2_score(y_test, y_pred_arvore)
mae_arvore = mean_absolute_error(y_test, y_pred_arvore)

print("Exercício 7 - Árvore de Regressão:")
print("-" * 35)
print(f"R² Árvore: {r2_arvore:.4f}")
print(f"MAE Árvore: {mae_arvore:.4f}")
print("-" * 35)
print(f"Mediana da Renda Real (Teste): R$ {mediana_real:.2f}")
print(f"Mediana da Renda Predita (Árvore): R$ {mediana_predita:.2f}")

Exercício 7 - Árvore de Regressão:
-----------------------------------
R² Árvore: 0.2198
MAE Árvore: 0.5659
-----------------------------------
Mediana da Renda Real (Teste): R$ 3456.45
Mediana da Renda Predita (Árvore): R$ 3315.09
